Use this script to tabulate simulation status for models in this batch.

In [2]:
from pathlib import Path
from tqdm import tqdm
import pandas
import re
import sqlite3

In [3]:
simfolder = Path(r'E:\SWHC049-06\residential measures\SWHC049-06 SEER Rated AC HP')

In [4]:

ENERGYPLUS_MODEL_FILEPATTERN = 'instance.idf'
ENERGYPLUS_OUT_FILEPATTERN = 'instance-out.sql'
ENERGYPLUS_ERR_FILEPATTERN = 'instance-out.err'

root = simfolder
#def gather_batch_stats(root: Path = '.'):
result = []
# Loop over subfolders and create progress bars.
for subroot in root.glob('*'):
    subroot_relpath = subroot.relative_to(root)
    # Count composed models.
    subroot_composed_models = list(subroot.glob('**/runs/**/'+ENERGYPLUS_MODEL_FILEPATTERN))
    n_models = len(subroot_composed_models)

    if n_models > 0:
        # Create the progress bar here.
        myiter = tqdm(subroot_composed_models, desc=subroot_relpath.parts[0])

        for i,f1 in enumerate(myiter):
            fileerr = f1.parent.joinpath(ENERGYPLUS_ERR_FILEPATTERN)
            fileout = f1.parent.joinpath(ENERGYPLUS_OUT_FILEPATTERN)

            # Parse the filename following DEER conventions
            # In 2023, residential models would be found like this:
            # relpath = "SFm_Furnace_1975\runs\CZ01\SFm&1&rDXGF&Ex&SpaceHtg_eq__GasFurnace\Msr-Res-GasFurnace-AFUE95-ECM\instance-out.sql
            # In 2024, commercial models would be found like this:
            # relpath = "SWXX000-00 Measure Name_1975\runs\CZ01\Asm\defaults\instance-out.sql"
            relpath = f1.relative_to(root)

            meas_group_vintage_combo, _, cz, cohort, techid, _ = relpath.parts
            cohort_clean = cohort.replace("&","=")
            meas_group, bldgvint = meas_group_vintage_combo.rsplit("_", 1)

            fileerr_exists = fileerr.exists()
            fileout_exists = fileout.exists()

            if not fileerr_exists:
                success = False
                timestamp = None
            else:
                # Match for a line at the beginning of the file like this:
                # Program Version,EnergyPlus, Version 9.5.0-de239b2e5f, YMD=2025.07.29 19:21,
                content = fileerr.read_text().splitlines()
                firstline = content[0]
                m = re.search(r'YMD=(?P<timestamp>[^,]*)',firstline)
                if not m:
                    timestamp = None
                else:
                    timestamp = m['timestamp']

                # Match for a line at the end of the file like this:
                # ************* EnergyPlus Completed Successfully-- 4 Warning; 0 Severe Errors; Elapsed Time=00hr 01min 53.20sec
                lastline = content[-1]
                success = (lastline.find(r'************* EnergyPlus Completed Successfully') > 0)
                m = re.search(r'-- (?P<warnings>\d*) Warning; (?P<errors>\d*) Severe Errors; Elapsed Time', lastline)
                if not m:
                    warnings = None
                    errors = None
                else:
                    warnings = m['warnings']
                    errors = m['errors']

            if not fileout_exists:
                fileout
            else:
                pass

            hastempdir = False
            for f3 in f1.parent.glob("instance*"):
                if f3.is_dir():
                    hastempdir = True

            result.append((relpath, meas_group_vintage_combo, cz, cohort, techid,
                           fileerr_exists, fileout_exists,
                           timestamp, success, warnings, errors, hastempdir))

        myiter.close()
    # end models
# end subroot folder
stats = pandas.DataFrame(result,
                          columns=['relpath', 'meas_group_vintage_combo', 'cz', 'cohort', 'techid',
                                   'fileerr_exists', 'fileout_exists',
                                   'timestamp', 'success', 'warnings', 'errors', 'hastempdir'])
stats

SWHC049-06 SEER Rated AC HP_SFm_New: 100%|██████████| 736/736 [00:04<00:00, 180.39it/s]


,relpath,meas_group_vintage_combo,cz,cohort,techid,fileerr_exists,fileout_exists,timestamp,success,warnings,errors,hastempdir
0,SWHC049-06 SEER Rated AC HP_DMo\runs\CZ01\DMo&...,SWHC049-06 SEER Rated AC HP_DMo,CZ01,DMo&0&rDXGF&Ex&dxAC_equip,dxAC-Res-SEER-13.0,True,True,2025.06.12 05:03,True,4,0,True
1,SWHC049-06 SEER Rated AC HP_DMo\runs\CZ01\DMo&...,SWHC049-06 SEER Rated AC HP_DMo,CZ01,DMo&0&rDXGF&Ex&dxAC_equip,dxAC-Res-SEER-14.0,True,True,2025.06.12 05:03,True,4,0,True
2,SWHC049-06 SEER Rated AC HP_DMo\runs\CZ01\DMo&...,SWHC049-06 SEER Rated AC HP_DMo,CZ01,DMo&0&rDXGF&Ex&dxAC_equip,dxAC-Res-SEER-14.5,True,True,2025.06.12 05:03,True,4,0,True
3,SWHC049-06 SEER Rated AC HP_DMo\runs\CZ01\DMo&...,SWHC049-06 SEER Rated AC HP_DMo,CZ01,DMo&0&rDXGF&Ex&dxAC_equip,dxAC-Res-SEER-15.0,True,True,2025.06.12 05:03,True,4,0,True
4,SWHC049-06 SEER Rated AC HP_DMo\runs\CZ01\DMo&...,SWHC049-06 SEER Rated AC HP_DMo,CZ01,DMo&0&rDXGF&Ex&dxAC_equip,dxAC-Res-SEER-16.0,True,True,2025.06.12 05:03,True,4,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
3003,SWHC049-06 SEER Rated AC HP_SFm_New\runs\CZ16\...,SWHC049-06 SEER Rated AC HP_SFm_New,CZ16,SFm&2&rDXHP&New&dxHP_equip,HSPF_9p4_SEER_17_Msr,True,True,2025.04.30 16:37,True,94,0,False
3004,SWHC049-06 SEER Rated AC HP_SFm_New\runs\CZ16\...,SWHC049-06 SEER Rated AC HP_SFm_New,CZ16,SFm&2&rDXHP&New&dxHP_equip,HSPF_9p5_SEER_18_Msr,True,True,2025.04.30 16:31,True,94,0,False
3005,SWHC049-06 SEER Rated AC HP_SFm_New\runs\CZ16\...,SWHC049-06 SEER Rated AC HP_SFm_New,CZ16,SFm&2&rDXHP&New&dxHP_equip,HSPF_9p5_SEER_19_Msr,True,True,2025.04.30 16:27,True,94,0,False
3006,SWHC049-06 SEER Rated AC HP_SFm_New\runs\CZ16\...,SWHC049-06 SEER Rated AC HP_SFm_New,CZ16,SFm&2&rDXHP&New&dxHP_equip,Res-GasFurnace-AFUE80-ECM,True,True,2025.04.30 16:00,True,53,0,False


In [5]:
stats[stats['success']==False]

,relpath,meas_group_vintage_combo,cz,cohort,techid,fileerr_exists,fileout_exists,timestamp,success,warnings,errors,hastempdir


In [6]:
stats[stats['hastempdir']==True]

,relpath,meas_group_vintage_combo,cz,cohort,techid,fileerr_exists,fileout_exists,timestamp,success,warnings,errors,hastempdir
0,SWHC049-06 SEER Rated AC HP_DMo\runs\CZ01\DMo&...,SWHC049-06 SEER Rated AC HP_DMo,CZ01,DMo&0&rDXGF&Ex&dxAC_equip,dxAC-Res-SEER-13.0,True,True,2025.06.12 05:03,True,4,0,True
1,SWHC049-06 SEER Rated AC HP_DMo\runs\CZ01\DMo&...,SWHC049-06 SEER Rated AC HP_DMo,CZ01,DMo&0&rDXGF&Ex&dxAC_equip,dxAC-Res-SEER-14.0,True,True,2025.06.12 05:03,True,4,0,True
2,SWHC049-06 SEER Rated AC HP_DMo\runs\CZ01\DMo&...,SWHC049-06 SEER Rated AC HP_DMo,CZ01,DMo&0&rDXGF&Ex&dxAC_equip,dxAC-Res-SEER-14.5,True,True,2025.06.12 05:03,True,4,0,True
3,SWHC049-06 SEER Rated AC HP_DMo\runs\CZ01\DMo&...,SWHC049-06 SEER Rated AC HP_DMo,CZ01,DMo&0&rDXGF&Ex&dxAC_equip,dxAC-Res-SEER-15.0,True,True,2025.06.12 05:03,True,4,0,True
4,SWHC049-06 SEER Rated AC HP_DMo\runs\CZ01\DMo&...,SWHC049-06 SEER Rated AC HP_DMo,CZ01,DMo&0&rDXGF&Ex&dxAC_equip,dxAC-Res-SEER-16.0,True,True,2025.06.12 05:03,True,4,0,True
6,SWHC049-06 SEER Rated AC HP_DMo\runs\CZ01\DMo&...,SWHC049-06 SEER Rated AC HP_DMo,CZ01,DMo&0&rDXGF&Ex&dxAC_equip,dxAC-Res-SEER-18.0,True,True,2025.06.12 05:03,True,4,0,True
15,SWHC049-06 SEER Rated AC HP_DMo\runs\CZ01\DMo&...,SWHC049-06 SEER Rated AC HP_DMo,CZ01,DMo&0&rDXGF&New&dxAC_equip,dxAC-Res-SEER-15.0,True,True,2025.06.12 05:07,True,4,0,True
16,SWHC049-06 SEER Rated AC HP_DMo\runs\CZ01\DMo&...,SWHC049-06 SEER Rated AC HP_DMo,CZ01,DMo&0&rDXGF&New&dxAC_equip,dxAC-Res-SEER-16.0,True,True,2025.06.12 05:07,True,4,0,True
27,SWHC049-06 SEER Rated AC HP_DMo\runs\CZ01\DMo&...,SWHC049-06 SEER Rated AC HP_DMo,CZ01,DMo&0&rDXHP&Ex&dxHP_equip,HSPF_8p0_SEER_14_Std,True,True,2025.06.12 05:05,True,2138,0,True
28,SWHC049-06 SEER Rated AC HP_DMo\runs\CZ01\DMo&...,SWHC049-06 SEER Rated AC HP_DMo,CZ01,DMo&0&rDXHP&Ex&dxHP_equip,HSPF_8p2_SEER_14.5_Std,True,True,2025.06.12 05:05,True,2138,0,True


In [7]:
stats.to_csv('simulation_stats.csv')
